# Importing Libraries

In [ ]:
import pandas as pd # type: ignore

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Loading Data

In [ ]:
EventsAlertsDataset = []

Labels = []

# TODO: Fill Desired Server Name

ServerName = ""

EventsAlertsDatasetFilePath = f"../Datasets/AIT-LDS V1.1/data/mail.{ServerName}.com/suricata/eve.json"

LabelsFilePath = f"../Datasets/AIT-LDS V1.1/data/mail.{ServerName}.com/suricata/eve.json"

In [ ]:
from Utils.OnCallLibs import JsonLib

with open(EventsAlertsDatasetFilePath, "r") as EventsAlertsDatasetFile:

	while (True):
		
		EventAlert = EventsAlertsDatasetFile.readline()

		if not EventAlert:

			break

		EventsAlertsDataset.append(JsonLib.loads(EventAlert))


In [ ]:
with open(LabelsFilePath, "r") as LabelsFile:

	while (True):

		Label = LabelsFile.readline()

		if not Label:

			break

		Labels.append(Label.split(",")[0])

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Lines Matching

In [ ]:
LabelRLEList = []

CurrentMatchingattackStatus = Labels[0]

CurrentCount = 0

for Label in Labels:

	PossibleNewAttackStatus = Label

	if PossibleNewAttackStatus == CurrentMatchingattackStatus:

		CurrentCount += 1

	else:

		LabelRLEList.append({"Key" : str(CurrentMatchingattackStatus), "Value" : str(CurrentCount)})

		CurrentMatchingattackStatus = PossibleNewAttackStatus

		CurrentCount = 1

# Adding Last Match
LabelRLEList.append({"Key" : str(CurrentMatchingattackStatus), "Value" : str(CurrentCount)})


In [ ]:
AttackStatusCorrespondingEvents = []

LeftIndex = 0

RightIndex = 0

for RLE in LabelRLEList:

	RightIndex += int(RLE["Value"])

	AttackStatusCorrespondingEvents.append({"AttackStatus" :  RLE["Key"], "CorrespondingEvents" : EventsAlertsDataset[LeftIndex : RightIndex], "EventCount" : RLE["Value"], "AlertCount" : 0})

	LeftIndex = RightIndex

In [ ]:
del EventsAlertsDataset
del Labels

# Generating Mini Report

In [ ]:
for AttackStatusEvents in AttackStatusCorrespondingEvents:
	
	AlertCount = 0

	for Event in AttackStatusEvents["CorrespondingEvents"]:
		
		if Event["event_type"] == "alert":

			AlertCount += 1

	
	AttackStatusEvents["AlertCount"] = AlertCount

In [ ]:

AttackStatuList = []

EventCountList = []

AlertCountList = []

for AttackStatusEvents in AttackStatusCorrespondingEvents:

	AttackStatuList.append(AttackStatusEvents["AttackStatus"])

	EventCountList.append(AttackStatusEvents["EventCount"])
	
	AlertCountList.append(AttackStatusEvents["AlertCount"])

In [ ]:
ReportDataFrame = pd.DataFrame(list(zip(AttackStatuList, EventCountList, AlertCountList)), columns = ["Attack Status", "Corresponding Events Generated", "Corresponding Alerts Generated"])

In [ ]:
ReportDataFrame.to_csv("SpiralMiniReport.csv", index = False)